# Creación de un modelo
Una vez vistas las posibilidades que ofrece, se va a modificar la librería para poder crear un entorno con sus acciones y agentes personalizado, para comprobar de una forma más extensa las funcionalidades de la librería.<br>
Basado en el juego del Trex de Chrome y en la implementación de un entorno personalizado (https://blog.paperspace.com/creating-custom-environments-openai-gym/) 

In [1]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random

from gym import Env, spaces
import time

font = cv2.FONT_HERSHEY_COMPLEX_SMALL 

In [2]:
class ChromeTrex(Env):
    def __init__(self):
        super(ChromeTrex, self).__init__()
        

        
        # Definición del espacio de observaciones (límites)
        self.observation_shape = (600, 800, 3)
        self.observation_space = spaces.Box(low = np.zeros(self.observation_shape), 
                                            high = np.ones(self.observation_shape),
                                            dtype = np.float16)
    
        
        # Definición del espacio de acciones
        self.action_space = spaces.Discrete(4,)
                        
        #Creación del canvas
        self.canvas = np.ones(self.observation_shape) * 1
        
        self.elements = []
        

        # Área en la que podrá operar el trex
        self.y_min = 350
        self.x_min = 20
        self.y_max = 500
        self.x_max = 20

    def draw_elements_on_canvas(self):
        self.canvas = np.ones(self.observation_shape) * 1

        # Dibujo de los iconos en el canvas
        for elem in self.elements:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon

        text = 'Recompensas: {}'.format(self.ep_return)

        # Info en el canvas 
        self.canvas = cv2.putText(self.canvas, text, (10,20), font,  
                   0.8, (0,0,0), 1, cv2.LINE_AA)

    def reset(self):

        # Reseteo
        self.ep_return  = 0
        self.bird_count = 0

        # Posicion inicial del trex
        x = 20
        y = 400

        # Inicialización del Trex y elementos
        self.trex = Trex("Trex", self.x_max, self.x_min, self.y_max, self.y_min)
        self.trex.set_position(x,y)
        self.elements = [self.trex]

        self.canvas = np.ones(self.observation_shape) * 1

        # Dibujo de todo
        self.draw_elements_on_canvas()


        #Observación
        return self.canvas 

    def render(self, mode = "human"):
        #Dos tipos de renderizado, si se pone otro excepción
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Game", self.canvas)
            cv2.waitKey(10)

        elif mode == "rgb_array":
            return self.canvas

    def close(self):
        cv2.destroyAllWindows()
        
    def get_action_meanings(self):
        assert self.action_space.contains(action), "Invalid Action"        
        return {0: "Down", 1: "Up", 2: "Do Nothing"}

       
    def has_collided(self, elem1, elem2): #Comprobar las posiciones
        x_col = False
        y_col = False

        elem1_x, elem1_y = elem1.get_position()
        elem2_x, elem2_y = elem2.get_position()

        if 2 * abs(elem1_x - elem2_x) <= (elem1.icon_w + elem2.icon_w):
            x_col = True

        if 2 * abs(elem1_y - elem2_y) <= (elem1.icon_h + elem2.icon_h):
            y_col = True

        if x_col and y_col:
            return True

        return False
    
    def step(self, action):
        done = False

        # Accion inválida
        assert self.action_space.contains(action), "Invalid Action"

        # Recompensa
        reward = 1      

        # Se aplica la acción
        if action == 0:
            self.trex.move(0,100)
        elif action == 1:
            self.trex.move(0,-100)
        elif action == 2:
            self.trex.move(0,0)

        # Si es menor, aparece un pajaro*
        if random.random() < 0.01:

            # Aparece un pájaro (2 posibles lugares de aparición)
            bird_y = 350 if random.randint(0, 1) == 0 else 400
            spawned_bird = Bird("bird_{}".format(self.bird_count),  800, 0, bird_y,bird_y)
            self.bird_count += 1

 
            bird_x = 800            
            spawned_bird.set_position(800, bird_y)

            # Añade el pájaro a los elementos que se encuentran en el entorno 
            self.elements.append(spawned_bird)    


        for elem in self.elements:
            if isinstance(elem, Bird):
                # Si el pájaro ha llegado al final, se elimina del entorno
                if elem.get_position()[0] <= self.x_min:
                    self.elements.remove(elem)
                else:
                    # Mueve el pajaro a la izquierda (avance)
                    elem.move(-5,0)

                # Si se choca el pájaro con el trex, se elimina
                if self.has_collided(self.trex, elem):
                    done = True
                    reward = -10
                    self.elements.remove(self.trex)


        self.ep_return += 1

        # Se dibuja en el canvas
        self.draw_elements_on_canvas()


        return self.canvas, reward, done, []

In [3]:
class Point(object): #Clase genérica para los elementos del canvas
                     #(lo unico que cambia es la imagen ya que ambos son puntos en el espacio)
    def __init__(self, name, x_max, x_min, y_max, y_min):
        self.x = 0
        self.y = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.name = name
    
    def set_position(self, x, y):
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h)
    
    def get_position(self):
        return (self.x, self.y)
    
    def move(self, del_x, del_y):
        self.x += del_x
        self.y += del_y
        
        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)

    def clamp(self, n, minn, maxn):
        return max(min(maxn, n), minn)

In [4]:
class Trex(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Trex, self).__init__(name, x_max, x_min, y_max, y_min)
        #Generación de la imagen y redimensión
        self.icon = cv2.imread("./media/trex1.png") / 255.0
        self.icon_w = 64
        self.icon_h = 64
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

    
class Bird(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Bird, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("./media/bird.jpg") / 255.0
        self.icon_w = 25 
        self.icon_h = 25
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        

In [5]:
from IPython import display

env = ChromeTrex()
obs = env.reset()
total = 0

for episode in range(20):
    obs = env.reset()
    for t in range(1000):
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)

        env.render()

        if done == True:
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()

/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Episodio terminado después de 158 timesteps
Episodio terminado después de 176 timesteps
Episodio terminado después de 152 timesteps
Episodio terminado después de 333 timesteps
Episodio terminado después de 293 timesteps
Episodio terminado después de 209 timesteps
Episodio terminado después de 299 timesteps
Episodio terminado después de 182 timesteps
Episodio terminado después de 152 timesteps
Episodio terminado después de 232 timesteps
Episodio terminado después de 200 timesteps
Episodio terminado después de 238 timesteps
Episodio terminado después de 153 timesteps
Episodio terminado después de 265 timesteps
Episodio terminado después de 158 timesteps
Episodio terminado después de 181 timesteps
Episodio terminado después de 354 timesteps
Episodio terminado después de 248 timesteps
Episodio terminado después de 252 timesteps
Episodio terminado después de 246 timesteps
Tiempo medio: 224.05


In [19]:
env.close()